# Analyzing the CIA Factbook Data Using SQL

In this project, we'll query the data from the CIA World Factbook, a dataset which provides basic information on all of the countries on Earth.

First we connect to the database.

In [7]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

We then look at the different tables located in the database.

In [4]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


We can see that there are two different tables, 'sqlite_sequence' and 'facts'. We can see that 'facts' is the table with the data we want to query, so we will reference this table for the rest of the project.

Let's take a quick glimpse at the 'facts' table.

In [6]:
%%sql 
SELECT *
FROM facts
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


We have 11 different variables. 
* name — the name of the country.
* area — the country's total area (both land and water).
* area_land — the country's land area in square kilometers.
* area_water — the country's water area in square kilometers.
* population — the country's population.
* population_growth — the country's population growth as a percentage.
* birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
* death_rate — the country's death rate, or the number of death per year per 1,000 people.

We'll next see what the highest and lowest populations and population growth rates are.

In [9]:
%%sql
SELECT  MIN(population) AS min_pop,
        MAX(population) AS max_pop,
        MIN(population_growth) AS min_pop_growth,
        MAX(population_growth) AS max_pop_growth
    FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


There shouldn't be a country with a population of 0 or a population with over 7 billion, so we should further query the data to see where these outliers come from.

In [5]:
%%sql
SELECT * 
FROM facts 
WHERE population == (SELECT MIN(population)
              FROM facts
              );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [6]:
%%sql
SELECT * 
FROM facts 
WHERE population == (SELECT MAX (population)
                    FROM facts
                    );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


We have two rows, one labeled 'Antarctica', with a population of 0, and one labeled 'World', listed with the world's entire population. For the purpose of this project, which is to make insights about individual countries, we can ignore these two rows entirely.

In [19]:
%%sql
SELECT MIN(population) AS min_pop,
        MAX(population) AS max_pop,
        MIN(population_growth) AS min_pop_growth,
        MAX(population_growth) AS max_pop_growth
    FROM facts
    WHERE name <> 'World' AND name <> 'Antarctica'

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
48,1367485388,0.0,4.02


When we filter out these outliers, we can see that the smallest population listed is 48, with the biggest being over 1.3 billion. The highest rate of population growth is 4.02, and the lowest listed being 0, even with our outliers removed.

In [35]:
%%sql
SELECT *
FROM facts
WHERE name <> 'World'
ORDER BY population DESC
LIMIT 2

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04


The two most populous countries are China followed by India. We can see that although China has the higher population, India has the higher rate of growth, a sign that India will soon overtake China as the highest populated country. This is backed by statistics from the U.N. showing that 42.7% of the Indian population is under 25, compared to China which only show 34.1%  being under 25.

We'll list the countries with a rate of population growth of 0.

In [21]:
%%sql 
SELECT *
FROM facts
WHERE population_growth == 0.0

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
190,vt,Holy See (Vatican City),0,0,0,842,0.0,None,None,None
200,ck,Cocos (Keeling) Islands,14,14,0,596,0.0,None,None,None
207,gl,Greenland,2166086,2166086,None,57733,0.0,14.48,8.49,5.98
238,pc,Pitcairn Islands,47,47,0,48,0.0,None,None,None


As we can see, 3 of these countries are small enough that they do not have listed birth, death, or migration rates. Greenland is the only country on Earth with a large enough population as to include birth, death, and migration rates and still have a rate of population growth be at 0.

We'll look at the average population and average area of all countries.

In [22]:
%%sql
SELECT AVG(population) AS avg_population,
        AVG(area) AS avg_area
    FROM facts
    WHERE name <> 'World' AND name <> 'Antarctica';

Done.


avg_population,avg_area
32377011.0125,555093.546184739


Now we'll build on this by querying for countries that are densely populated. We filter for countries that have a population which is ABOVE average and an area that is BELOW average.

In [25]:
%%sql
SELECT * , population/area AS pop_density
FROM facts
WHERE population > (SELECT AVG(population)
                   FROM facts
                   WHERE name <> 'World' AND name <> 'Antarctica'
                   )
    AND area < (SELECT AVG (area)
               FROM facts
               WHERE name <> 'World' AND name <> 'Antarctica'
               )
    ORDER BY pop_density DESC;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,pop_density
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46,1138
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0,492
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09,336
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0,335
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3,284
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54,263
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24,226
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1,205
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74,153
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0,132


Here we've created a new variable for population density, calculated by dividing the total population by the area. We can see that the country with the highest population density is Bangladesh, which is over double the second highest country, South Korea.

In [32]:
%%sql 
SELECT AVG(population_growth) AS avg_pop_growth
FROM facts
WHERE name <> 'World' AND name <> 'Antarctica'

Done.


avg_pop_growth
1.2014893617021278


The average population growth for the data is ~1.2. If we refer to our previous query, we can see that many of our most densely populated countries are well below the average for rate of population growth, especially South Korea, Japan, Germany, Italy, Thailand, and Poland. We can predict that these countries will begin to decline in population, due to their low rate of growth.

In [8]:
%%sql
SELECT name, population_growth, birth_rate, death_rate
FROM facts
ORDER BY population_growth DESC
LIMIT 10

Done.


name,population_growth,birth_rate,death_rate
South Sudan,4.02,36.91,8.18
Malawi,3.32,41.56,8.41
Burundi,3.28,42.01,9.27
Niger,3.25,45.45,12.42
Uganda,3.24,43.79,10.69
Qatar,3.07,9.84,1.53
Burkina Faso,3.03,42.03,11.72
Mali,2.98,44.99,12.89
Cook Islands,2.95,14.33,8.03
Iraq,2.93,31.45,3.77


The ten countries with the highest rate of population growth are listed above. Although most countries listed have a relatively large birth rate when compared to their death rate, one country in particular, 'Cook Islands', has a birth rate not much higher than their death rate.

In [17]:
%%sql 
SELECT *
FROM facts
WHERE population_growth == 2.95

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
219,cw,Cook Islands,236,236,0,9838,2.95,14.33,8.03,None


The total population of this country is a lot smaller than average, making it so their rate of population growth can be so high without a large gap between birth and death rates.

We expect countries that have a higher death rate than their birth rates to have a low rate of population growth.

In [23]:
%%sql
SELECT name, population_growth, birth_rate, death_rate, migration_rate
FROM facts
WHERE death_rate > birth_rate
ORDER BY death_rate DESC
LIMIT 10

Done.


name,population_growth,birth_rate,death_rate,migration_rate
Ukraine,0.6,10.72,14.46,2.25
Bulgaria,0.58,8.92,14.44,0.29
Latvia,1.06,10.0,14.31,6.26
Lithuania,1.04,10.1,14.27,6.27
Russia,0.04,11.6,13.69,1.69
Serbia,0.46,9.08,13.66,0.0
Belarus,0.2,10.7,13.36,0.7
Hungary,0.22,9.16,12.73,1.33
Moldova,1.03,12.0,12.59,9.67
Estonia,0.55,10.51,12.4,3.6


In [24]:
%%sql 
SELECT AVG(migration_rate) AS avg_mig_rate
FROM facts

Done.


avg_mig_rate
3.433946188340806


Most countries do follow this pattern, but we can see that countries with a migration rate that is much larger than the average overcome this pattern and still have a positive rate of population growth.